# MCU Scripts

## Configuration

In [1]:
%load_ext autotime

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import os

In [3]:
import re

## Analysis

### About the Data

#### Processed Scripts

In [4]:
lines = pd.read_csv('../clean_data/mcu_data.csv', index_col=0).reset_index(drop=True)[['character', 'line', 'movie', 'year', 'words']]
print('Entries: ', len(lines))
lines.head()
processed_movies = lines.groupby(['movie', 'year']).head(1)[['movie', 'year']].sort_values(['movie']).reset_index(drop=True)
processed_movies

Entries:  6509


,movie,year
0,Ant-Man,2015
1,Avengers: Age of Ultron,2015
2,Avengers: Endgame,2019
3,Avengers: Infinity War,2018
4,Captain America: Civil War,2016
5,Captain America: The First Avenger,2011
6,Captain America: The Winter Soldier,2014
7,Captain Marvel,2019
8,Iron Man,2008
9,Iron Man 2,2010


#### Raw Scripts

In [5]:
df = pd.read_csv('../raw_data/mcu_scipts.csv', index_col=0).merge(processed_movies, right_on=['movie'], left_on=['title'], how='left')
df['script_length'] = df['script'].apply(len)
print('Entries: ', len(df))
df.head(23)

Entries:  23


,title,script,movie,year,script_length
0,Ant-Man,Previous transcript:\n Next transcript:\n\n\n ...,Ant-Man,2015.0,82517
1,Ant-Man and the Wasp,This transcript is not finished!This page does...,NaN,NaN,67957
2,The Avengers,This transcript isn't tidy!This page's transcr...,The Avengers,2012.0,163543
3,Avengers: Age of Ultron,Previous transcript:\n Next transcript:\n\n\n ...,Avengers: Age of Ultron,2015.0,91399
4,Avengers: Endgame,Previous transcript:\n Next transcript:\n\n\n ...,Avengers: Endgame,2019.0,138200
5,Avengers: Infinity War,Previous transcript:\n Next transcript:\n\n\n ...,Avengers: Infinity War,2018.0,141191
6,Black Panther,This transcript isn't tidy!This page's transcr...,NaN,NaN,201332
7,Captain America: Civil War,Previous transcript:\n Next transcript:\n\n\n ...,Captain America: Civil War,2016.0,127046
8,Captain America: The First Avenger,Previous transcript:\n Next transcript:\n\n\n ...,Captain America: The First Avenger,2011.0,71770
9,Captain America: The Winter Soldier,Previous transcript:\n Next transcript:\n\n\n ...,Captain America: The Winter Soldier,2014.0,98173


In [17]:
script = df.script.values[1]
print(script[:1000])

This transcript is not finished!This page does not have the entire transcript.It is recommended to add the complete transcript.

This article is a stub. You can help Transcripts Wiki by  expanding it.     



 Previous transcript:
 Next transcript:


 Avengers: Infinity War
 Captain Marvel

Hank: [voiceover] I still think about the night your mother and I had to leave you...
[The camera cuts to a young Hope van Dyne, who is walking out of her bedroom into the hallway. There, Janet van Dyne and Hank Pym are talking to a woman.]
Janet: [to the woman, faintly] Hopefully, it's not for long, but I'll call you when we get settled.When I have a better indication of what's going on.
Young Hope: Mommy?
[Janet and Hank turn around to face their daughter. Janet sighs as Hank gets the bags situated.]
Janet: Oh, Jellybean. [She kneels down to meet eye-level with Hope.] Daddy and I have a last minute business trip, so Rose is going to stay with you for a few days.
Young Hope: No. I don't want you to

# Cleaning Up Script

## Find Actual Start of Script

In [18]:
script[:292]

'This transcript is not finished!This page does not have the entire transcript.It is recommended to add the complete transcript.\n\nThis article is a stub. You can help Transcripts Wiki by  expanding it.     \n\n\n\n Previous transcript:\n Next transcript:\n\n\n Avengers: Infinity War\n Captain Marvel\n\n'

In [19]:
script = script[292:]
print(script[:300])

Hank: [voiceover] I still think about the night your mother and I had to leave you...
[The camera cuts to a young Hope van Dyne, who is walking out of her bedroom into the hallway. There, Janet van Dyne and Hank Pym are talking to a woman.]
Janet: [to the woman, faintly] Hopefully, it's not for long


## Find Characters

In [23]:
characters = np.unique(re.findall(string=script, pattern='\n((?:[A-Z][a-z]+[\-\.]? ?)+):'), return_counts=True)
characters = pd.DataFrame(zip(characters[0], characters[1]), columns=['character', 'line_count'])
print(len(characters))
characters.sort_values(['line_count'], ascending=False)

20


,character,line_count
15,Scott,80
8,Hope,53
6,Hank,43
12,Luis,26
2,Cassie,19
17,Sonny,18
9,Janet,10
16,Scott Lang,7
13,Maggie,7
18,Woo,6


## Remove Narration

In [10]:
script_lines_only = re.sub(string=script, pattern='(\[.*\])\n?', repl='')
script_lines_only = re.sub(string=script_lines_only, pattern='\n(\[.*\]?)\n', repl='\n')
script_lines_only = re.sub(string=script_lines_only, pattern='\n(\(.*\)?)\n', repl='\n')
script_lines_only = re.sub(string=script_lines_only, pattern='\xa0', repl=' ')

print(script_lines_only[:1000])

Hank:  I still think about the night your mother and I had to leave you...
Janet:  Hopefully, it's not for long, but I'll call you when we get settled.When I have a better indication of what's going on.
Young Hope: Mommy?
Janet: Oh, Jellybean.  Daddy and I have a last minute business trip, so Rose is going to stay with you for a few days.
Young Hope: No. I don't want you to go.
Janet: Ugh! It's gonna be so boring! I won't be able to keep my... eyes...
Hank: Goodbye, sweetheart. Alright, we'll see you soon.  
Hank:  I wish we could have put down our bags, and tucked you back into your bed, but too many lives were at stake. 
Hank: Oh my god! They've already launched!
Janet: We have to stop it! Come on!
Hank:  The only way to disarm the missile was to get inside it, but the plating was too thick. The only way in was to shrink between the molecules. 
Janet: I can't get through! Hank...
Hank:  We both knew that if you go that small, there's no coming back. 
Hank:  It should have been me, bu

## Charactater Line Extractions

In [24]:
lines = re.findall(string=script_lines_only, pattern='((?:[A-Z][a-z]+[\-\.]? ?)+):\s*(.*)')
lines = pd.DataFrame(lines, columns=['character', 'line'])
lines['line'] = [x.rstrip().lstrip() for x in lines['line']]
lines['character'].value_counts()

Scott          79
Hope           53
Hank           43
Luis           26
Cassie         18
Sonny          18
Janet          10
Scott Lang      7
Woo             6
Maggie          6
Kurt            5
Paxton          5
Cassie Lang     5
Dave            3
Young Hope      3
Ava             3
Scott           2
Ant-Man         2
Video           1
Hanks           1
Hope            1
Ghost           1
Jimmy Woo       1
Name: character, dtype: int64